In [ ]:
import sys
print(f'Python: {sys.version}')

In [ ]:
from pprint import pprint
import math
import itertools

import numpy as np
import pandas as pd
import scipy
from scipy import signal, fft, datasets
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Fourier Transform

In [ ]:
def visualize_fourier_analysis(
    freqs, 
    sample_rate, 
    duration, 
    phase = 0, 
    constant = 0,
    normalize = False,
    logy=False, 
    add_alias=False,
    window_func=None,
):
    if isinstance(freqs, (int, float)):
        freqs = list(freqs)
    freqs = np.array(freqs)
    n_samples = int(sample_rate * duration)
    sample_spacing = 1/sample_rate
    n_periods = duration * freqs
    n_samples_per_period = n_samples/n_periods
    f_nyquist = 0.5*sample_rate

    t = np.linspace(0, duration, n_samples, endpoint=False)
    t_full = np.linspace(0, duration, 1000, endpoint=False)
    
    y = np.zeros_like(t)
    y_ideal = np.zeros_like(t_full)
    y_alias = np.zeros_like(t_full)
    peak_freq = freqs[0] # First frequency has largest amplitude
    freqs_alias = np.zeros_like(freqs)
    for idx, f in enumerate(freqs,1):
        y += (1/idx) * np.cos(2*np.pi*f*t + phase) + constant
        y_ideal += (1/idx) * np.cos(2*np.pi*f*t_full + phase) + constant
        
        if f < f_nyquist:
            diff = f_nyquist - f
            f_alias = f_nyquist + diff
            phase_alias = -phase
        elif f == f_nyquist:
            f_alias = f_nyquist + sample_rate
            phase_alias = phase
        else:
            f_mod = f % sample_rate
            phase_alias = -phase if f_mod > f_nyquist else phase
            diff = np.abs(f_mod - f_nyquist)
            f_alias = f_nyquist - diff
        freqs_alias[idx-1] = f_alias
        y_alias += (1/idx) * np.cos(2*np.pi*f_alias*t_full + phase_alias) + constant
    
    if normalize:
        y /= len(y)
        y_ideal /= len(y_ideal)
        y_alias /= len(y_alias)
    if window_func is not None:
        y *= window_func(len(y))
        y_ideal *= window_func(len(y_ideal))
        y_alias *= window_func(len(y_alias))

    fft_coef = fft.rfft(y)
    fft_mag = np.abs(fft_coef)
    fft_phase = np.angle(fft_coef)
    fft_freq = fft.rfftfreq(n_samples, sample_spacing)
    freq_bin_width = np.unique(np.diff(fft_freq))[0] # always 1 / duration
    freq_bin_prec = math.ceil(np.log10(duration))+1
    freq_bin_width_str = str(round(freq_bin_width, freq_bin_prec))
    f_nyquist_str = str(round(f_nyquist, freq_bin_prec))

    y_shift = fft_mag[0] / n_samples
    fft_peak_idx = np.argmax(fft_mag[1:]) + 1 # Ignore 0Hz freq
    fft_peak_mag = fft_freq[fft_peak_idx]
    fft_peak_phase = fft_phase[fft_peak_idx]
    
    y_fft_peak = np.cos(2*np.pi*fft_peak_mag*t_full + fft_peak_phase) + y_shift

    if normalize:
        y_fft_peak /= len(y_fft_peak)
    if window_func is not None:
        y_fft_peak *= window_func(len(y_fft_peak))
    ################################################################################
    # Plotting
    fig, axs = plt.subplots(4, figsize=(10,7))
    ax = axs[0]
    freqs_str = '+'.join([f'{f}Hz' for f in freqs])
    ax.plot(t_full, y_ideal, ':', label=f'Truth [{freqs_str}]')
    if add_alias:
        freqs_alias_str = '+'.join([f'{f}Hz' for f in freqs_alias])
        ax.plot(t_full, y_alias, ':', label=f'Alias [{freqs_alias_str}]')
    ax.scatter(t, y, color='k', marker='o', s=10, label=f'Sampled Data [{sample_rate}Hz]')
    ax.axhline(0, c='k')
    ax.set_title(f'Sampled Data [N={n_samples}]')
    ax.set_xlabel('Time (s)')
    ax.grid()
    ax.legend(loc='upper right')

    ax = axs[1]
    ax.scatter(fft_freq, fft_mag, marker='x', s=50)
    if logy:
        ax.set_yscale("log")
    for idx, f in enumerate(freqs):
        lw = 3 if idx == 0 else 1
        label = f'Truth [{peak_freq}Hz]' if idx==0 else None
        ax.axvline(f, c='g', lw=lw, label=label)
    ax.axvline(fft_peak_mag, c='k', ls=':', label=f'FFT Peak [{fft_peak_mag:.1f}Hz]')

    bin_width = round(freq_bin_width, math.ceil(np.log10(duration)))
    
    ax.set_title(f'Fourier Transform Amplitudes [$f_{{Nyquist}}$= {f_nyquist_str}Hz; Bin width={freq_bin_width_str:}Hz; FFT Peak={fft_peak_mag:.1f}Hz]')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Amplitude')
    ax.grid()
    ax.legend(loc='upper right')
    
    ax = axs[2]
    ax.scatter(fft_freq, fft_phase/np.pi, marker='x', s=50)
    ax.axvline(freqs[0], color='g', ls=':')
    ax.scatter(freqs[0], phase/np.pi, c='g', marker='+', s=200, label=f'Truth [{phase/np.pi}$\pi$ rad]')
    ax.scatter(fft_peak_mag, fft_peak_phase/np.pi, marker='o', facecolors='none', edgecolors='k', label=f'FFT Peak [{fft_peak_phase/np.pi:.1f}$\pi$ rad]')
    ax.set_title('Fourier Transform Phase')
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Phase ($\pi$ rad)')
    ax.set_ylim(-1.1, 1.1)
    ax.grid()
    ax.legend(loc='upper right')  

    ax = axs[3]
    ax.plot(t,y, lw=3, label='Data')
    ax.plot(t_full, y_fft_peak, label=f'FFT Peak [{fft_peak_mag:.1f}Hz]')
    ax.set_title('Data vs FFT Peak Frequency')
    ax.set_xlabel('Time (s)')
    ax.grid()
    ax.legend(loc='upper right')

    summary = f'Samples per period = {n_samples_per_period}'
    fig.tight_layout()

What explains these features of the discrete fourier transform below?
* The range of frequencies considered (i.e. nyquist frequency)?
* The spacing between frequencies (i.e. frequency bin width)?
* The amplitude of the peak?

What is the impact of
* Adding a constant
* Changing the phase
* A signal frequency being above the nyquist frequency?
* The data containing a non-integer number of signal periods?

In [ ]:
visualize_fourier_analysis(
    freqs       = [1], # Hz
    sample_rate = 10, # Hz
    duration    = 5, # seconds
    #constant    = 1,
    #phase       = 0.5 * np.pi, # radians
    #normalize   = True,
    #add_alias   = True,
    #logy        = True,
    #window_func = signal.windows.blackman
)

Future directions
1. Handling complex input data
1. Handling increased data dimenstions: N-dimensional Fourier Transforms
1. Handling different basis functions for the transform: Discrete Sin/Cosine Transforms
1. Handling non-uniformly spaced data: nonuniform discrete Fourier transform (NUDFT or NDFT), Hankel Transform, Least-squares spectral analysis 
1. Handling periodic and non-periodic noise (e.g. filtering, smoothing)
1. Handle non-periodic signals: Waveletes

# Scipy Signal Processing

## All module methods/attributes

In [ ]:
# pprint([x for x in dir(signal) if not x.startswith('_')])

################################################################################
# Convolution and Correlation
# signal.convolve()
# signal.convolve2d()
# signal.deconvolve()
# signal.fftconvolve()
# signal.oaconvolve()
# signal.sepfir2d

# signal.correlate()
# signal.correlate2d()
# signal.correlation_lags()

################################################################################
# Windows (scipy.signal.windows)
windows = {
    'signal.barthann()',
    'signal.bartlett()',
    'signal.blackman()',
    'signal.blackmanharris()',
    'signal.bohman()',
    'signal.boxcar()',
    'signal.chebwin()',
    'signal.cosine()',
    'signal.exponential()',
    'signal.flattop()',
    'signal.gaussian()',
    'signal.general_gaussian()',
    'signal.get_window()',
    'signal.hamming()',
    'signal.hann()',
    'signal.kaiser()',
    'signal.nuttall()',
    'signal.parzen()',
    'signal.triang()',
    'signal.tukey()',
}
# signal.windows.kaiser_bessel_derived()
# signal.windows.general_cosine()
# signal.windows.lanczos()
# signal.windows.dpss()
# signal.windows.taylor()
# signal.windows.general_hamming()

################################################################################
# Difference equation filtering 
# signals.lfilter()
# signals.lfilter_zi()
# signals.lfiltic()

################################################################################
# Unsorted
# signal.BadCoefficients
# signal.CZT
# signal.StateSpace
# signal.TransferFunction
# signal.ZerosPolesGain
# signal.ZoomFFT
# signals.abcd_normalize()
# signals.argrelextrema()
# signals.argrelmax()
# signals.argrelmin()
# signals.band_stop_obj()
# signals.bessel()
# signals.besselap()
# signals.bilinear()
# signals.bilinear_zpk()
# signals.bode()
# signals.bspline()
# signals.bsplines()
# signals.buttap()
# signals.butter()
# signals.buttord()
# signals.cascade()
# signals.cheb1ap()
# signals.cheb1ord()
# signals.cheb2ap()
# signals.cheb2ord()
# signals.cheby1()
# signals.cheby2()
# signals.check_COLA()
# signals.check_NOLA()
# signals.chirp()
# signals.choose_conv_method()
# signals.cmplx_sort()
# signals.coherence()
# signals.cont2discrete()
# signals.csd()
# signals.cspline1d()
# signals.cspline1d_eval()
# signals.cspline2d()
# signals.cubic()
# signals.cwt()
# signals.czt()
# signals.czt_points()
# signals.daub()
# signals.dbode()
# signals.decimate()
# signals.detrend()
# signals.dfreqresp()
# signals.dimpulse()
# signals.dlsim()
# signals.dlti()
# signals.dstep()
# signals.ellip()
# signals.ellipap()
# signals.ellipord()
# signals.filter_design()
# signals.filtfilt()
# signals.find_peaks()
# signals.find_peaks_cwt()
# signals.findfreqs()
# signals.fir_filter_design()
# signals.firls()
# signals.firwin()
# signals.firwin2()
# signals.freqresp()
# signals.freqs()
# signals.freqs_zpk()
# signals.freqz()
# signals.freqz_zpk()
# signals.gammatone()
# signals.gauss_spline()
# signals.gausspulse()
# signals.group_delay()
# signals.hilbert()
# signals.hilbert2()
# signals.iircomb()
# signals.iirdesign()
# signals.iirfilter()
# signals.iirnotch()
# signals.iirpeak()
# signals.impulse()
# signals.impulse2()
# signals.invres()
# signals.invresz()
# signals.istft()
# signals.kaiser_atten()
# signals.kaiser_beta()
# signals.kaiserord()
# signals.lombscargle()
# signals.lp2bp()
# signals.lp2bp_zpk()
# signals.lp2bs()
# signals.lp2bs_zpk()
# signals.lp2hp()
# signals.lp2hp_zpk()
# signals.lp2lp()
# signals.lp2lp_zpk()
# signals.lsim()
# signals.lsim2()
# signals.lti()
# signals.lti_conversion()
# signals.ltisys()
# signals.max_len_seq()
# signals.medfilt()
# signals.medfilt2d()
# signals.minimum_phase()
# signals.morlet()
# signals.morlet2()
# signals.normalize()
# signals.order_filter()
# signals.peak_prominences()
# signals.peak_widths()
# signals.periodogram()
# signals.place_poles()
# signals.qmf()
# signals.qspline1d()
# signals.qspline1d_eval()
# signals.qspline2d()
# signals.quadratic()
# signals.remez()
# signals.resample()
# signals.resample_poly()
# signals.residue()
# signals.residuez()
# signals.ricker()
# signals.savgol_coeffs()
# signals.savgol_filter()
# signals.sawtooth()
# signals.signaltools()
# signals.sos2tf()
# signals.sos2zpk()
# signals.sosfilt()
# signals.sosfilt_zi()
# signals.sosfiltfilt()
# signals.sosfreqz()
# signals.spectral()
# signals.spectrogram()
# signals.spline()
# signals.spline_filter()
# signals.square()
# signals.ss2tf()
# signals.ss2zpk()
# signals.step()
# signals.step2()
# signals.stft()
# signals.sweep_poly()
# signals.symiirorder1()
# signals.symiirorder2()
# signals.test()
# signals.tf2sos()
# signals.tf2ss()
# signals.tf2zpk()
# signals.unique_roots()
# signals.unit_impulse()
# signals.upfirdn()
# signals.vectorstrength()
# signals.waveforms()
# signals.wavelets()
# signals.welch()
# signals.wiener()
# signals.zoom_fft()
# signals.zpk2sos()
# signals.zpk2ss()
# signals.zpk2tf()

## B-Splines

# Windows

# Filters

Terms
* Filter - any operation modifying an input signal
* Types of filters
    * Linearity
        * Linear - representable with matrix multiplication
        * Non-Linear
    * Shift-invariance
    * Time-discrete filters
        * Finite response (FIR)
        * Infinite response (IIR)

## `windows`

In [ ]:
window_functions = [
    f for f in dir(signal.windows) 
    if not (f.startswith('_') or f in ['get_window', 'windows'])
]
print(window_functions)

In [ ]:
n = math.ceil(math.sqrt(len(window_functions)))
fig, axs = plt.subplots(n,n, figsize=(3*n, 3*n))
N = 101
for i, window in enumerate(window_functions):
    row, col = divmod(i, n)
    ax = axs[row, col]
    ax.set_title(window.title())
    if window == 'kaiser':
        window_arg = (window, 5) # beta    
    elif window == 'kaiser_bessel_derived':
        pass
    elif window == 'gaussian':
        window_arg = (window, 7) # standard deviation
    elif window == 'general_cosine':
        HFT90D = [1, 1.942604, 1.340318, 0.440811, 0.043097]
        window_arg = (window, HFT90D) # weighting_coefficients
    elif window == 'general_gaussian':
        window_arg = (window, 1.5, 7) # power, width
    elif window == 'general_hamming':
        window_arg = (window, 0.7) # window_coefficient
    elif window == 'dpss':
        window_arg = (window, 3) # normalized_half_bandwidth
    elif window == 'chebwin':
        window_arg = (window, 100) # attenuation
    else:
        window_arg = window
    
    if window == 'kaiser_bessel_derived':
        # Doesn't work with get_window for some reason
        ax.plot(signal.windows.kaiser_bessel_derived(N-1, 5))
    else:
        ax.plot(signal.get_window(window_arg, N))
    
    ax.set_xlim(-10, N+10)
    ax.set_ylim(-0.1,1.1)
    ax.grid()
fig.tight_layout()
    

## `convolve` and `correlate`

In [ ]:
# What is a good example to highlight the speed difference of fft vs direct?
# When to use convolve vs correlate?

mode_opts = ['full', 'same', 'valid']
method_opts = ['fft', 'direct'] # method = 'fft' => fftconvolve
n_convolutions = 1

sig = np.repeat([0,1,0], N)
filt = np.zeros(N)

i_start = math.ceil(N * (1/4))
i_end = math.floor(N * (3/4))
filt[i_start:i_end] = np.linspace(0,1,i_end-i_start)

opts = list(itertools.product(mode_opts, method_opts))
n_rows = len(opts)+1
fig, axs = plt.subplots(n_rows,1,figsize=(5, n_rows*3))

for idx, (mode, method) in enumerate(opts):
    sig_conv = sig.copy()
    norm = 1/filt.sum()
    for i in range(n_convolutions):
        sig_conv = signal.convolve(sig_conv, filt, mode, method)
    
    ax = axs[idx+1]
    ax.set_title(f'Mode = {mode}; Method = {method}')
    ax.plot(sig_conv)
    ax.grid()

ax = axs[0]
ax.plot(sig, label='Signal')
ax.plot(filt, label='Filter')
ax.grid()
ax.legend()
fig.tight_layout()

## `lfilter`

Difference-equation filtering

In [ ]:
signal.lfilter()

## Spectral Analysis

## Short-time Fourier Transform

## Detrend

# Spectral Analysis

In [ ]:
fs = 10e3
N = 10**5
amp = 2*np.sqrt(2)
freq = 1270.0
noise_power = 0.001 * fs / 2
time = np.arange(N) / fs
x = amp*np.sin(2*np.pi*freq*time)
x += np.random.normal(scale=np.sqrt(noise_power), size=time.shape)

# Periodogram
f, Pper_spec = signal.periodogram(x, fs, 'flattop', scaling='spectrum')
# Full FFT Transform
fft_coef = fft.rfft(x)
fft_freq = fft.rfftfreq(N, 1/fs)
fft_amp  = np.abs(fft_coef) / N
fft_density = 2 * fft_amp**2

plt.plot(f, Pper_spec, label='Periodogram', linewidth=4)
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD')
plt.grid()
plt.plot(fft_freq, fft_density, label='FFT Power')
plt.semilogy()
plt.legend()
plt.show()


# Detrend

# Other